seed 42

In [ ]:
# [Cell 1] Imports & Config
import os, gc, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.impute import KNNImputer
import catboost as cb

try:
    import optuna
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    HAS_OPTUNA = True
except ImportError:
    HAS_OPTUNA = False
    print("[INFO] optuna 미설치 → 기본 파라미터 사용")

# -------------------------
# Paths
# -------------------------
DATA_DIR = "../data"
OUT_DIR  = "../outputs"
os.makedirs(OUT_DIR, exist_ok=True)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"

# -------------------------
# CV / Seeds
# -------------------------
N_FOLDS_TRAIN = 20
N_FOLDS_TUNE  = 5
N_FOLDS_TE    = 5

SEED = 42 
SEEDS = [42]
TUNE_SEED = 42           

OPTUNA_TRIALS = 40

# -------------------------
# GPU Auto Detect
# -------------------------
import subprocess
def has_gpu():
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
        return result.returncode == 0
    except:
        return False

TASK_TYPE = "GPU" if has_gpu() else "CPU"
print(f"[INFO] Device: {TASK_TYPE}")

# [Cell 2] Utils
def add_na_indicators(df, cols):
    for c in cols:
        if c in df.columns:
            df[f"{c}__isna"] = df[c].isna().astype(np.int8)
    return df

def safe_div(a, b, eps=1.0):
    return (a + eps) / (b + eps)

def clip_log1p(s, lo=None, hi=None):
    x = s.copy()
    if lo is not None: x = x.clip(lower=lo)
    if hi is not None: x = x.clip(upper=hi)
    return np.log1p(x)

def choose_smoothing(train_df, col, base=20):
    nunique = train_df[col].nunique(dropna=False)
    if nunique <= 10:   return base
    if nunique <= 50:   return base * 3
    if nunique <= 200:  return base * 8
    return base * 15

# [Cell 3] Imputation (DI rule-fill + continuous-only KNN)
def impute_missing(train_df, test_df):
    di_fill_cols = [
        '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '총 생성 배아 수', '이식된 배아 수',
        '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
        '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
        '난자 채취 경과일', '난자 해동 경과일', '배아 이식 경과일', '배아 해동 경과일',
        '난자 혼합 경과일', '임신 시도 또는 마지막 임신 경과 연수',
    ]

    for df in [train_df, test_df]:
        if '시술 유형' in df.columns:
            di_mask = df['시술 유형'] == 'DI'
            for col in di_fill_cols:
                if col in df.columns:
                    df.loc[di_mask, col] = df.loc[di_mask, col].fillna(0)

    # continuous-only KNN (binary flags 제외)
    knn_cols = [
        '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수',
        '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
        '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',
        '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
        '배아 이식 경과일', '난자 혼합 경과일', '배아 해동 경과일',
        '난자 채취 경과일', '난자 해동 경과일', '임신 시도 또는 마지막 임신 경과 연수',
    ]
    knn_cols = [c for c in knn_cols if c in train_df.columns]
    has_null = [c for c in knn_cols if train_df[c].isnull().any()]

    if has_null:
        print(f"  KNN Imputing {len(has_null)} continuous columns...")
        imputer = KNNImputer(n_neighbors=5, weights='distance')
        train_df[has_null] = imputer.fit_transform(train_df[has_null])
        test_df[has_null]  = imputer.transform(test_df[has_null])
        print("  Done.")

    return train_df, test_df

# [Cell 4] Feature Engineering
def preprocess(df):
    d = df.copy()

    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        x = str(x)
        if "IUI" in x:  return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x:  return "IVF"
        if "DI" in x:   return "DI"
        return "Other"

    d["시술_대분류"] = d["특정 시술 유형"].apply(major_procedure)
    d["BLASTOCYST_포함"] = d["특정 시술 유형"].astype(str).str.contains("BLASTOCYST", na=False).astype(np.int8)
    d["AH_포함"] = d["특정 시술 유형"].astype(str).str.contains("AH", na=False).astype(np.int8)

    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    exist_cols = [c for c in embryo_stage_cols if c in d.columns]
    d["배아_이식_여부"] = 1 - d[exist_cols].isna().all(axis=1).astype(np.int8)

    def embryo_stage(row):
        if row["배아_이식_여부"] == 0: return "배아단계_미도달"
        if pd.isna(row.get("총 생성 배아 수")) or row.get("총 생성 배아 수", 0) == 0: return "배아생성_실패"
        if pd.isna(row.get("이식된 배아 수")) or row.get("이식된 배아 수", 0) == 0: return "이식_미실시"
        return "이식_완료"

    d["배아_진행_단계"] = d.apply(embryo_stage, axis=1)

    def collapse_trials(x):
        if pd.isna(x): return "Unknown"
        if x == "0회": return "0회"
        if x in ["1회", "2회"]: return "1–2회"
        return "3회 이상"
    d["총시술_bin3"] = d["총 시술 횟수"].apply(collapse_trials)

    def age_group_simple(age):
        if pd.isna(age) or age == "알 수 없음": return "Unknown"
        if age == "만18-34세": return "34세 이하"
        if age in ["만35-37세", "만38-39세"]: return "35-39세"
        return "40세 이상"
    d["나이_3구간"] = d["시술 당시 나이"].apply(age_group_simple)

    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return "0개"
        if count <= 2: return "1-2개"
        return "3개 이상"
    d["이식배아_구간"] = d["이식된 배아 수"].apply(embryo_count_bin)

    d["Day5_이식_여부"] = (d["배아 이식 경과일"] == 5.0).astype(np.int8)

    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    icols = [c for c in infertility_cols if c in d.columns]
    d["불임_원인_개수"] = d[icols].sum(axis=1) if icols else 0

    d["배아_해동_실시_여부"] = d["배아 해동 경과일"].notna().astype(np.int8)

    # ratios: Laplace + log1p
    d["배아_이식_비율"] = clip_log1p(safe_div(d["이식된 배아 수"].fillna(0), d["총 생성 배아 수"].fillna(0)))
    d["배아_저장_비율"] = clip_log1p(safe_div(d["저장된 배아 수"].fillna(0), d["총 생성 배아 수"].fillna(0)))
    d["배아_생성_효율"] = clip_log1p(safe_div(d["총 생성 배아 수"].fillna(0), d["수집된 신선 난자 수"].fillna(0)))
    d["미세주입_생성_효율"] = clip_log1p(safe_div(d["미세주입에서 생성된 배아 수"].fillna(0), d["미세주입된 난자 수"].fillna(0)))
    d["난자_활용률"] = clip_log1p(safe_div(d["혼합된 난자 수"].fillna(0), d["수집된 신선 난자 수"].fillna(0)))

    # interactions
    d["나이×Day5"] = d["시술 당시 나이"].astype(str) + "_" + d["Day5_이식_여부"].astype(str)
    d["시술횟수×나이"] = d["총시술_bin3"].astype(str) + "_" + d["나이_3구간"].astype(str)
    d["나이×배아진행"] = d["시술 당시 나이"].astype(str) + "_" + d["배아_진행_단계"]
    d["시기코드×나이"] = d["시술 시기 코드"].astype(str) + "_" + d["나이_3구간"].astype(str)
    d["나이×단일이식"] = d["시술 당시 나이"].astype(str) + "_" + d["단일 배아 이식 여부"].fillna(-1).astype(int).astype(str)

    # culture time
    d["배양기간"] = d["배아 이식 경과일"] - d["난자 혼합 경과일"]

    # ordinal map
    ord_map = {"0회":0, "1회":1, "2회":2, "3회":3, "4회":4, "5회":5, "6회 이상":6}
    ord_pairs = [
        ("총시술_ord","총 시술 횟수"), ("IVF시술_ord","IVF 시술 횟수"),
        ("클리닉시술_ord","클리닉 내 총 시술 횟수"), ("DI시술_ord","DI 시술 횟수"),
        ("총임신_ord","총 임신 횟수"), ("IVF임신_ord","IVF 임신 횟수"),
        ("총출산_ord","총 출산 횟수"), ("IVF출산_ord","IVF 출산 횟수"),
        ("DI임신_ord","DI 임신 횟수"), ("DI출산_ord","DI 출산 횟수"),
    ]
    for newc, src in ord_pairs:
        if src in d.columns:
            d[newc] = d[src].map(ord_map)

    # egg age
    def get_egg_age(row):
        src = row.get("난자 출처")
        if src == "본인 제공":
            return row.get("시술 당시 나이")
        if src == "기증 제공":
            donor = row.get("난자 기증자 나이")
            return donor if (pd.notna(donor) and donor != "알 수 없음") else "만18-34세"
        return row.get("시술 당시 나이")
    d["난자_나이"] = d.apply(get_egg_age, axis=1)

    # embryo source
    def get_embryo_source(row):
        fr = int(row.get("동결 배아 사용 여부") or 0) if pd.notna(row.get("동결 배아 사용 여부")) else 0
        fs = int(row.get("신선 배아 사용 여부") or 0) if pd.notna(row.get("신선 배아 사용 여부")) else 0
        if fr and fs: return "both"
        if fr: return "frozen"
        if fs: return "fresh"
        return "none"
    d["배아_출처"] = d.apply(get_embryo_source, axis=1)
    d["시기×배아출처"] = d["시술 시기 코드"].astype(str) + "_" + d["배아_출처"]

    # embryo reason simple
    def embryo_reason_simple(x):
        if pd.isna(x): return "미도달"
        x = str(x)
        if "현재 시술용" in x: return "시술용"
        if "기증" in x: return "기증"
        if "연구" in x: return "연구"
        return "저장"
    d["배아생성이유"] = d["배아 생성 주요 이유"].apply(embryo_reason_simple)

    # ovulation stim × type
    if "배란 유도 유형" in d.columns:
        d["배란 유도 유형"] = d["배란 유도 유형"].replace({
            '생식선 자극 호르몬': '기록되지 않은 시행',
            '세트로타이드 (억제제)': '기록되지 않은 시행'
        })
    d["배란_자극_유도"] = d["배란 자극 여부"].astype(str) + "_" + d["배란 유도 유형"].astype(str)

    d["나이_알수없음"] = (d["시술 당시 나이"] == "알 수 없음").astype(np.int8)

    # male/female infertility
    male_inf = ["불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인",
                "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태"]
    female_inf = ["불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증"]
    male_cols = [c for c in male_inf if c in d.columns]
    fem_cols  = [c for c in female_inf if c in d.columns]
    d["남성_불임_수"] = d[male_cols].sum(axis=1) if male_cols else 0
    d["여성_불임_수"] = d[fem_cols].sum(axis=1) if fem_cols else 0
    denom = (d["남성_불임_수"] + d["여성_불임_수"]).replace(0, np.nan)
    d["여성_불임_비율"] = (d["여성_불임_수"] / denom).fillna(0.0)

    # history rates (Laplace + log1p)
    for c in ["IVF시술_ord","IVF임신_ord","IVF출산_ord","총시술_ord","총임신_ord","총출산_ord","클리닉시술_ord"]:
        if c in d.columns:
            d[c] = d[c].fillna(0)

    d["IVF_임신률"] = clip_log1p(safe_div(d["IVF임신_ord"], d["IVF시술_ord"]))
    d["IVF_출산률"] = clip_log1p(safe_div(d["IVF출산_ord"], d["IVF임신_ord"]))
    d["총_출산률"]  = clip_log1p(safe_div(d["총출산_ord"], d["총임신_ord"]))
    d["클리닉_비율"] = clip_log1p(safe_div(d["클리닉시술_ord"], d["총시술_ord"]))

    # extra
    d["미세주입_이식_비율"] = clip_log1p(safe_div(d["미세주입 배아 이식 수"].fillna(0), d["이식된 배아 수"].fillna(0)))
    d["혼합_생성률"] = clip_log1p(safe_div(d["총 생성 배아 수"].fillna(0), (d["혼합된 난자 수"].fillna(0) + d["해동 난자 수"].fillna(0))))
    d["미세주입_배아_생성률"] = clip_log1p(safe_div(d["미세주입에서 생성된 배아 수"].fillna(0), d["미세주입된 난자 수"].fillna(0)))
    d["총_사용_배아"] = d["해동된 배아 수"].fillna(0) + d["총 생성 배아 수"].fillna(0)
    d["IVF_정자_미혼합"] = ((d["파트너 정자와 혼합된 난자 수"].fillna(0) == 0) &
                         (d["기증자 정자와 혼합된 난자 수"].fillna(0) == 0) &
                         (d["시술 유형"] == "IVF")).astype(np.int8)

    # time interactions
    d["시기×단일이식"] = d["시술 시기 코드"].astype(str) + "_" + d["단일 배아 이식 여부"].fillna(0).astype(int).astype(str)
    d["시기×배란자극"] = d["시술 시기 코드"].astype(str) + "_" + d["배란 자극 여부"].astype(str)
    d["시기×유전진단"] = d["시술 시기 코드"].astype(str) + "_" + d["착상 전 유전 진단 사용 여부"].fillna(0).astype(int).astype(str)

    d["신선_배양시간"] = d["배아 이식 경과일"] - d["난자 혼합 경과일"]
    d["동결_배양시간"] = d["배아 이식 경과일"] - d["배아 해동 경과일"]
    d["이상적_배양"] = d["배아 이식 경과일"].isin([3.0, 5.0]).astype(np.int8)

    def fresh_egg_tier(row):
        if pd.isna(row.get("신선 배아 사용 여부")) or int(row.get("신선 배아 사용 여부") or 0) == 0:
            return "not_fresh"
        eggs = row.get("수집된 신선 난자 수")
        eggs = 0 if pd.isna(eggs) else eggs
        code = str(row.get("시술 시기 코드"))
        if eggs > 10: return f"{code}_E3"
        if eggs > 0:  return f"{code}_E2"
        return f"{code}_E1"
    d["시기별_신선난자_구간"] = d.apply(fresh_egg_tier, axis=1)

    def frozen_thaw_tier(row):
        if pd.isna(row.get("동결 배아 사용 여부")) or int(row.get("동결 배아 사용 여부") or 0) == 0:
            return "not_frozen"
        thawed = row.get("해동된 배아 수")
        thawed = 0 if pd.isna(thawed) else thawed
        code = str(row.get("시술 시기 코드"))
        if thawed > 3: return f"{code}_T3"
        if thawed > 0: return f"{code}_T2"
        return f"{code}_T1"
    d["시기별_해동배아_구간"] = d.apply(frozen_thaw_tier, axis=1)

    d["유전검사_합"] = d["착상 전 유전 진단 사용 여부"].fillna(0) + d["착상 전 유전 검사 사용 여부"].fillna(0)
    d["PGD_실시"] = d["PGD 시술 여부"].fillna(0).astype(int)
    d["PGS_실시"] = d["PGS 시술 여부"].fillna(0).astype(int)

    d["대리모 여부"] = d["대리모 여부"].fillna(-1)

    # NA indicators (핵심)
    na_cols = [
        "총 생성 배아 수","이식된 배아 수","저장된 배아 수",
        "수집된 신선 난자 수","혼합된 난자 수",
        "배아 이식 경과일","난자 혼합 경과일","배아 해동 경과일",
        "신선_배양시간","동결_배양시간","배양기간",
    ]
    d = add_na_indicators(d, na_cols)

    return d

# [Cell 5] Target Encoding (OOF 5-fold 고정, smoothing 자동)
def oof_target_encode(train_df, test_df, col, target_col, n_folds=5, seed=42, smoothing=20):
    global_mean = train_df[target_col].mean()
    train_enc = pd.Series(np.nan, index=train_df.index, dtype=float)
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for tr_idx, va_idx in skf.split(train_df, train_df[target_col]):
        tr = train_df.iloc[tr_idx]
        stats = tr.groupby(col)[target_col].agg(["mean", "count"])
        stats["enc"] = (stats["mean"] * stats["count"] + global_mean * smoothing) / (stats["count"] + smoothing)
        mapping = stats["enc"].to_dict()
        train_enc.iloc[va_idx] = train_df.iloc[va_idx][col].map(mapping).fillna(global_mean)

    stats = train_df.groupby(col)[target_col].agg(["mean", "count"])
    stats["enc"] = (stats["mean"] * stats["count"] + global_mean * smoothing) / (stats["count"] + smoothing)
    test_enc = test_df[col].map(stats["enc"].to_dict()).fillna(global_mean)

    return train_enc.values, test_enc.values

# [Cell 6] Optuna (seed=42에서만 1회)
def tune_catboost_optuna(X_train, y, cat_indices, task_type, n_trials=40, seed=42, n_folds=5):
    if not HAS_OPTUNA:
        print("[SKIP] Optuna 미설치")
        return None

    def objective(trial):
        params = {
            'iterations': 2500,
            'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.12, log=True),
            'depth': trial.suggest_int('depth', 5, 10),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 20.0, log=True),
            'random_strength': trial.suggest_float('random_strength', 0.1, 2.5),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'border_count': trial.suggest_int('border_count', 64, 255),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 80),
            'rsm': trial.suggest_float('rsm', 0.6, 1.0),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'random_seed': seed,
            'eval_metric': 'AUC',
            'loss_function': 'Logloss',
            'verbose': 0,
            'early_stopping_rounds': 120,
            'use_best_model': True,
            'task_type': task_type,
        }

        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        aucs = []

        for tr_idx, va_idx in skf.split(X_train, y):
            pool_tr = cb.Pool(X_train.iloc[tr_idx], y[tr_idx], cat_features=cat_indices)
            pool_va = cb.Pool(X_train.iloc[va_idx], y[va_idx], cat_features=cat_indices)

            model = cb.CatBoostClassifier(**params)
            model.fit(pool_tr, eval_set=pool_va)

            pred = model.predict_proba(X_train.iloc[va_idx])[:, 1]
            aucs.append(roc_auc_score(y[va_idx], pred))

        return float(np.mean(aucs))

    print(f"\n[Optuna] CatBoost 튜닝 (trials={n_trials}, folds={n_folds}, seed={seed})")
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    print(f"[Optuna] Best CV AUC: {study.best_value:.6f}")
    print(f"[Optuna] Best params: {study.best_params}")
    return study.best_params

# [Cell 7] One Run (single seed) - 20fold train/predict
def train_cb_oof(
    train_df, test_df, y,
    feature_cols, cat_cols, task_type,
    seed, cb_params_base,
    n_folds=20
):
    X_train = train_df[feature_cols].copy()
    X_test  = test_df[feature_cols].copy()

    for c in cat_cols:
        X_train[c] = X_train[c].astype(str).fillna("NA")
        X_test[c]  = X_test[c].astype(str).fillna("NA")

    cat_indices = [feature_cols.index(c) for c in cat_cols]

    cb_params = dict(cb_params_base)
    cb_params["random_seed"] = seed
    cb_params["task_type"]   = task_type

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    oof_pred = np.zeros(len(train_df), dtype=float)
    test_pred = np.zeros(len(test_df), dtype=float)
    fold_aucs = []
    fi_sum = np.zeros(len(feature_cols), dtype=float)

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train, y), 1):
        pool_tr = cb.Pool(X_train.iloc[tr_idx], y[tr_idx], cat_features=cat_indices)
        pool_va = cb.Pool(X_train.iloc[va_idx], y[va_idx], cat_features=cat_indices)

        model = cb.CatBoostClassifier(**cb_params)
        model.fit(pool_tr, eval_set=pool_va)

        oof_pred[va_idx] = model.predict_proba(X_train.iloc[va_idx])[:, 1]
        test_pred += model.predict_proba(X_test)[:, 1] / n_folds

        fauc = roc_auc_score(y[va_idx], oof_pred[va_idx])
        fold_aucs.append(fauc)

        # fold별 FI 누적(평균용)
        fi_sum += model.get_feature_importance(pool_tr)

        best_iter = getattr(model, "best_iteration_", None)
        print(f"  seed={seed} | fold {fold}/{n_folds} | AUC={fauc:.6f} | best_iter={best_iter}")

        gc.collect()

    oof_auc = roc_auc_score(y, oof_pred)
    fi_mean = fi_sum / n_folds
    

    return {
        "seed": seed,
        "oof_auc": oof_auc,
        "fold_mean": float(np.mean(fold_aucs)),
        "fold_std": float(np.std(fold_aucs)),
        "oof_pred": oof_pred,
        "test_pred": test_pred,
        "fi_mean": fi_mean
    }

# [Cell 8] Main Notebook Run (Seed별 TE 재생성 버전)
# Load → Impute → FE(공통) → [Seed loop: TE 재생성 → Train 20-fold] → Ensemble

print("="*80)
print("12v3 Notebook (B안): DI+KNN → FE → [TE per seed] → 20-fold × Seeds → Ensemble")
print("="*80)

# -------------------------
# 0) Load
# -------------------------
train_raw = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_raw  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
sub       = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

# -------------------------
# 1) Imputation (공통 1회)
# -------------------------
print("\n[Step 1] Imputation...")
train_raw, test_raw = impute_missing(train_raw, test_raw)

# -------------------------
# 2) Feature Engineering (공통 1회)
# -------------------------
print("\n[Step 2] Feature Engineering...")
train_fe = preprocess(train_raw)
test_fe  = preprocess(test_raw)

y = train_fe[TARGET_COL].astype(int).values

# drop / base feature cols (TE 추가 전 기준)
drop_cols = [ID_COL, TARGET_COL]
base_feature_cols = [c for c in train_fe.columns if c not in drop_cols]

# cat cols (TE 추가 전 기준)
base_cat_cols = [c for c in base_feature_cols if str(train_fe[c].dtype) in ["object", "category"]]

print(f"\nBase Features: {len(base_feature_cols)} (cat={len(base_cat_cols)}, num={len(base_feature_cols)-len(base_cat_cols)})")

# -------------------------
# 3) Optuna (딱 1회) - TE 포함된 데이터로 튜닝하는게 더 맞음
#    -> TUNE_SEED로 TE를 한 번 만들고 그걸로 튜닝
# -------------------------
te_cols = [
    "시술 시기 코드", "특정 시술 유형", "시술 당시 나이",
    "배아_진행_단계", "시술_대분류",
    "시기×배아출처", "배란_자극_유도",
    "시기×단일이식", "시기×유전진단",
    "시술횟수×나이", "나이×배아진행",
    "시기별_신선난자_구간", "시기별_해동배아_구간",
]

def apply_te_for_seed(train_df, test_df, seed_for_te):
    """train_df/test_df에 TE__* 컬럼을 생성해서 리턴 (원본 변경 X)"""
    tr = train_df.copy()
    te = test_df.copy()
    for col in te_cols:
        if col not in tr.columns:
            continue
        sm = choose_smoothing(tr, col, base=20)
        tr_enc, te_enc = oof_target_encode(
            tr, te, col, TARGET_COL,
            n_folds=N_FOLDS_TE, seed=seed_for_te, smoothing=sm
        )
        tr[f"TE__{col}"] = tr_enc
        te[f"TE__{col}"] = te_enc
    return tr, te

print("\n[Step 3] Optuna 준비 (TUNE_SEED로 TE 1회 생성)...")
train_tune, test_tune = apply_te_for_seed(train_fe, test_fe, seed_for_te=TUNE_SEED)

# 최종 feature/cat cols (TE 포함 기준)
feature_cols = [c for c in train_tune.columns if c not in drop_cols]
cat_cols = [c for c in feature_cols if str(train_tune[c].dtype) in ["object", "category"]]
print(f"Features(+TE): {len(feature_cols)} (cat={len(cat_cols)}, num={len(feature_cols)-len(cat_cols)})")

print("\n[Step 4] Optuna (once)...")
best_params = None
if HAS_OPTUNA and OPTUNA_TRIALS > 0:
    X_tune = train_tune[feature_cols].copy()

    # CatBoost 입력: cat은 str
    for c in cat_cols:
        X_tune[c] = X_tune[c].astype(str).fillna("NA")

    # 안전장치: inf 방지
    X_tune.replace([np.inf, -np.inf], np.nan, inplace=True)

    cat_indices = [feature_cols.index(c) for c in cat_cols]
    best_params = tune_catboost_optuna(
        X_tune, y, cat_indices,
        task_type=TASK_TYPE,
        n_trials=OPTUNA_TRIALS,
        seed=TUNE_SEED,
        n_folds=N_FOLDS_TUNE
    )
else:
    print("[Optuna] skip")

# -------------------------
# 4) Base params (호환/안정 버전)
# - Bayesian + bagging_temperature 조합
# - subsample은 제거(혼선/제약 피하기)
# -------------------------
cb_params_base = {
    "iterations": 5000,
    "learning_rate": 0.05,
    "depth": 7,
    "l2_leaf_reg": 6.0,
    "random_strength": 1.0,
    "bagging_temperature": 0.2,
    "border_count": 180,
    "rsm": 0.85,

    "bootstrap_type": "Bayesian",

    "eval_metric": "AUC",
    "loss_function": "Logloss",
    "verbose": 300,
    "early_stopping_rounds": 200,
    "use_best_model": True,
}
if best_params:
    # Optuna 결과 merge (불필요/충돌 가능 파라미터가 섞이면 제거)
    cb_params_base.update(best_params)
    # Bayesian에서는 subsample 혼선 가능 → 혹시 들어오면 제거
    cb_params_base.pop("subsample", None)

print("\n[Final Params] CatBoost base params:")
print(cb_params_base)

# -------------------------
# 5) Seed loop: seed별 TE 재생성 + 20-fold 학습
# -------------------------
print("\n[Step 5] Train 20-fold × Seeds (TE per seed)...")

results = []
oof_ens = np.zeros(len(train_fe), dtype=float)
test_ens = np.zeros(len(test_fe), dtype=float)
fi_ens = np.zeros(len(feature_cols), dtype=float)

for s in SEEDS:
    print("\n" + "-"*70)
    print(f"Seed {s} run (TE seed={s}, CV seed={s})")
    print("-"*70)

    # seed별 TE 생성
    train_s, test_s = apply_te_for_seed(train_fe, test_fe, seed_for_te=s)

    # train_cb_oof는 내부에서 cat str 처리함.
    # 대신 여기서 inf 방지 한 번 더 안전하게 적용
    train_s.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_s.replace([np.inf, -np.inf], np.nan, inplace=True)

    res = train_cb_oof(
        train_s, test_s, y,
        feature_cols, cat_cols,
        task_type=TASK_TYPE,
        seed=s,
        cb_params_base=cb_params_base,
        n_folds=N_FOLDS_TRAIN
    )

    results.append(res)
    oof_ens += res["oof_pred"] / len(SEEDS)
    test_ens += res["test_pred"] / len(SEEDS)
    fi_ens  += res["fi_mean"] / len(SEEDS)

# -------------------------
# 6) Ensemble Summary
# -------------------------
oof_auc_ens = roc_auc_score(y, oof_ens)
print("\n" + "="*70)
print("Seed-wise summary")
for r in results:
    print(f" seed={r['seed']} | OOF_AUC={r['oof_auc']:.6f} | fold_mean={r['fold_mean']:.6f} ± {r['fold_std']:.6f}")
print(f"\n >>> Ensemble OOF AUC = {oof_auc_ens:.6f}")
print("="*70)

# -------------------------
# 7) Save artifacts
# -------------------------
tag = f"12v3B_cb_ens_{oof_auc_ens:.6f}_S{'-'.join(map(str, SEEDS))}_F{N_FOLDS_TRAIN}_TE{N_FOLDS_TE}"

oof_path = os.path.join(OUT_DIR, f"{tag}_oof.npy")
test_path = os.path.join(OUT_DIR, f"{tag}_test.npy")
np.save(oof_path, oof_ens)
np.save(test_path, test_ens)

# submission
sub2 = sub.copy()
sub2["probability"] = test_ens
sub_path = os.path.join(OUT_DIR, f"{tag}_submit.csv")
sub2.to_csv(sub_path, index=False)

# feature importance (ensemble mean)
fi_df = pd.DataFrame({"feature": feature_cols, "importance": fi_ens}).sort_values("importance", ascending=False)
fi_path = os.path.join(OUT_DIR, f"{tag}_fi.csv")
fi_df.to_csv(fi_path, index=False)

# params log
summary_path = os.path.join(OUT_DIR, f"{tag}_summary.txt")
with open(summary_path, "w", encoding="utf-8") as f:
    f.write("12v3 B안 (Seed별 TE 재생성) CatBoost Ensemble\n")
    f.write("="*70 + "\n")
    f.write(f"DEVICE: {TASK_TYPE}\n")
    f.write(f"SEEDS: {SEEDS}\n")
    f.write(f"N_FOLDS_TRAIN: {N_FOLDS_TRAIN}\n")
    f.write(f"N_FOLDS_TE: {N_FOLDS_TE}\n")
    f.write(f"N_FOLDS_TUNE: {N_FOLDS_TUNE}\n")
    f.write(f"OOF_AUC_ENSEMBLE: {oof_auc_ens:.6f}\n\n")
    f.write("[Seed results]\n")
    for r in results:
        f.write(f" seed={r['seed']} | OOF_AUC={r['oof_auc']:.6f} | fold_mean={r['fold_mean']:.6f} ± {r['fold_std']:.6f}\n")
    f.write("\n[CatBoost params]\n")
    f.write(str(cb_params_base) + "\n")
    f.write("\n[Optuna best_params]\n")
    f.write(str(best_params) + "\n")
    f.write("\n[Top 30 FI]\n")
    f.write(fi_df.head(30).to_string(index=False) + "\n")

print("\nSaved:")
print(" -", sub_path)
print(" -", oof_path)
print(" -", test_path)
print(" -", fi_path)
print(" -", summary_path)

seed 42 결과 저장

In [ ]:
# ====== SAVE ONLY SEED=42 RESULTS (after seed 42 finished) ======
import os, numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

print("[OUT_DIR]", os.path.abspath(OUT_DIR))

# 1) results 안에서 seed=42 결과 찾기
res42 = None
for r in results:
    if r["seed"] == 42:
        res42 = r
        break

if res42 is None:
    raise ValueError("seed=42 결과가 results에 없습니다.)")

oof_42  = res42["oof_pred"]
test_42 = res42["test_pred"]
auc_42  = roc_auc_score(y, oof_42)

print(f"[OK] Seed 42 OOF AUC = {auc_42:.6f}")
print(f"fold_mean={res42['fold_mean']:.6f} ± {res42['fold_std']:.6f}")

tag = f"seed42_only_F{N_FOLDS_TRAIN}_TE{N_FOLDS_TE}_AUC{auc_42:.6f}"

# 2) submission 저장
sub_out = sub.copy()
sub_out["probability"] = test_42
sub_path = os.path.join(OUT_DIR, f"{tag}_submit.csv")
sub_out.to_csv(sub_path, index=False)

# 3) oof/test npy 저장
np.save(os.path.join(OUT_DIR, f"{tag}_oof.npy"),  oof_42)
np.save(os.path.join(OUT_DIR, f"{tag}_test.npy"), test_42)

# 4) summary 저장
with open(os.path.join(OUT_DIR, f"{tag}_summary.txt"), "w", encoding="utf-8") as f:
    f.write("CatBoost seed=42 only (partial run)\n")
    f.write("="*60 + "\n")
    f.write(f"DEVICE: {TASK_TYPE}\n")
    f.write(f"N_FOLDS_TRAIN: {N_FOLDS_TRAIN}\n")
    f.write(f"N_FOLDS_TE: {N_FOLDS_TE}\n")
    f.write(f"Seed 42 OOF AUC: {auc_42:.6f}\n")
    f.write(f"fold_mean: {res42['fold_mean']:.6f} ± {res42['fold_std']:.6f}\n")
    f.write(f"CatBoost params: {cb_params_base}\n")
    f.write(f"Optuna best_params: {best_params}\n")
    f.write(f"len(results): {len(results)}\n")

print("\n[SAVED]")
print(" -", sub_path)

best_param 저장

In [ ]:
# best param 저장 
import json, os

# best_params는 dict (optuna 결과)
param_path = os.path.join(OUT_DIR, "optuna_best_params_seed42.json")
with open(param_path, "w", encoding="utf-8") as f:
    json.dump(best_params, f, ensure_ascii=False, indent=2)

print("[SAVED] ", param_path)
print(best_params)

seed 202, 777 추가 학습

In [ ]:
# ============================================================
# Step 5 only: Train remaining seeds (202, 777)
# + optionally load saved seed42 outputs and make ensemble
# ============================================================

import os, numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

# ---- settings ----
OUT_DIR = "../outputs"
N_FOLDS_TRAIN = 20
N_FOLDS_TE = 5

DONE_SEEDS = [42]                 # 이미 끝난 seed
RUN_SEEDS  = [202, 777]           # 지금 추가로 돌릴 seed
ALL_SEEDS_FOR_ENSEMBLE = [42, 202, 777]  # 최종 앙상블에 포함할 seed

SEED42_TAG_PREFIX = "seed42_only_F20_TE5_AUC"  

def find_seed42_paths(out_dir):
    """outputs 폴더에서 seed42_only_F20_TE5_AUC*.npy 찾기"""
    oof = None
    test = None
    for fn in os.listdir(out_dir):
        if fn.startswith(SEED42_TAG_PREFIX) and fn.endswith("_oof.npy"):
            oof = os.path.join(out_dir, fn)
        if fn.startswith(SEED42_TAG_PREFIX) and fn.endswith("_test.npy"):
            test = os.path.join(out_dir, fn)
    return oof, test

print("\n[Step 5 ONLY] Train remaining seeds:", RUN_SEEDS)

results_new = []
oof_dict = {}
test_dict = {}

# --- (A) 먼저 seed42 결과 로드 ---
seed42_oof_path, seed42_test_path = find_seed42_paths(OUT_DIR)
if seed42_oof_path and seed42_test_path:
    oof_dict[42]  = np.load(seed42_oof_path)
    test_dict[42] = np.load(seed42_test_path)
    print(f"[LOAD] seed42 oof:  {seed42_oof_path}")
    print(f"[LOAD] seed42 test: {seed42_test_path}")
else:
    print("[WARN] seed42 저장 파일을 outputs에서 못찾음. (앙상블은 202/777만으로 계산됨)")

# --- (B) 202,777 학습 ---
for s in RUN_SEEDS:
    print("\n" + "-"*70)
    print(f"Seed {s} run (TE seed={s}, CV seed={s})")
    print("-"*70)

    # seed별 TE 생성 (train_fe/test_fe는 이미 만들어져 있어야 함)
    train_s, test_s = apply_te_for_seed(train_fe, test_fe, seed_for_te=s)

    # inf 방지
    train_s.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_s.replace([np.inf, -np.inf], np.nan, inplace=True)

    res = train_cb_oof(
        train_s, test_s, y,
        feature_cols, cat_cols,
        task_type=TASK_TYPE,
        seed=s,
        cb_params_base=cb_params_base,
        n_folds=N_FOLDS_TRAIN
    )
    results_new.append(res)

    # seed별 저장
    tag_s = f"seed{s}_F{N_FOLDS_TRAIN}_TE{N_FOLDS_TE}_AUC{res['oof_auc']:.6f}"
    oof_path  = os.path.join(OUT_DIR, f"{tag_s}_oof.npy")
    test_path = os.path.join(OUT_DIR, f"{tag_s}_test.npy")
    sub_path  = os.path.join(OUT_DIR, f"{tag_s}_submit.csv")

    np.save(oof_path,  res["oof_pred"])
    np.save(test_path, res["test_pred"])

    sub_s = sub.copy()
    sub_s["probability"] = res["test_pred"]
    sub_s.to_csv(sub_path, index=False)

    print(f"[SAVED seed={s}]")
    print(" -", oof_path)
    print(" -", test_path)
    print(" -", sub_path)

    oof_dict[s]  = res["oof_pred"]
    test_dict[s] = res["test_pred"]

# --- (C) 앙상블 (가능한 seed만 평균) ---
available_seeds = [s for s in ALL_SEEDS_FOR_ENSEMBLE if s in oof_dict]
print("\n[Ensemble] available seeds:", available_seeds)

oof_ens  = np.zeros_like(next(iter(oof_dict.values())))
test_ens = np.zeros_like(next(iter(test_dict.values())))

for s in available_seeds:
    oof_ens  += oof_dict[s]  / len(available_seeds)
    test_ens += test_dict[s] / len(available_seeds)

oof_auc_ens = roc_auc_score(y, oof_ens)
print(f" >>> Ensemble OOF AUC = {oof_auc_ens:.6f}")

# ensemble 저장
tag_ens = f"cb_ens_{oof_auc_ens:.6f}_S{'-'.join(map(str, available_seeds))}_F{N_FOLDS_TRAIN}_TE{N_FOLDS_TE}"

ens_oof_path  = os.path.join(OUT_DIR, f"{tag_ens}_oof.npy")
ens_test_path = os.path.join(OUT_DIR, f"{tag_ens}_test.npy")
ens_sub_path  = os.path.join(OUT_DIR, f"{tag_ens}_submit.csv")
ens_sum_path  = os.path.join(OUT_DIR, f"{tag_ens}_summary.txt")

np.save(ens_oof_path,  oof_ens)
np.save(ens_test_path, test_ens)

sub_ens = sub.copy()
sub_ens["probability"] = test_ens
sub_ens.to_csv(ens_sub_path, index=False)

with open(ens_sum_path, "w", encoding="utf-8") as f:
    f.write("CatBoost Ensemble (partial incremental run)\n")
    f.write("="*70 + "\n")
    f.write(f"DEVICE: {TASK_TYPE}\n")
    f.write(f"SEEDS_USED: {available_seeds}\n")
    f.write(f"N_FOLDS_TRAIN: {N_FOLDS_TRAIN}\n")
    f.write(f"N_FOLDS_TE: {N_FOLDS_TE}\n")
    f.write(f"OOF_AUC_ENSEMBLE: {oof_auc_ens:.6f}\n\n")
    f.write("[cb_params_base]\n")
    f.write(str(cb_params_base) + "\n")

print("\n[ENSEMBLE SAVED]")
print(" -", ens_sub_path)
print(" -", ens_oof_path)
print(" -", ens_test_path)
print(" -", ens_sum_path)


seed 1024, 2048, 4096 추가 학습

In [ ]:
# ============================================================
# Step 5 only: Train remaining seeds (1024, 2048, 4096)
# + optionally load saved seed42 outputs and make ensemble
# ============================================================

import os, numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

# ---- settings ----
OUT_DIR = "../outputs"
N_FOLDS_TRAIN = 20
N_FOLDS_TE = 5

DONE_SEEDS = [42, 202, 777]                 # 이미 끝난 seed
RUN_SEEDS  = [1024, 2048, 4096]           # 지금 추가로 돌릴 seed
ALL_SEEDS_FOR_ENSEMBLE = [42, 202, 777]  # 최종 앙상블에 포함할 seed

SEED42_TAG_PREFIX = "seed42_only_F20_TE5_AUC"  

def find_seed42_paths(out_dir):
    """outputs 폴더에서 seed42_only_F20_TE5_AUC*.npy 찾기"""
    oof = None
    test = None
    for fn in os.listdir(out_dir):
        if fn.startswith(SEED42_TAG_PREFIX) and fn.endswith("_oof.npy"):
            oof = os.path.join(out_dir, fn)
        if fn.startswith(SEED42_TAG_PREFIX) and fn.endswith("_test.npy"):
            test = os.path.join(out_dir, fn)
    return oof, test

print("\n[Step 5 ONLY] Train remaining seeds:", RUN_SEEDS)

results_new = []
oof_dict = {}
test_dict = {}

# --- (A) 먼저 seed42 결과 로드(있으면) ---
seed42_oof_path, seed42_test_path = find_seed42_paths(OUT_DIR)
if seed42_oof_path and seed42_test_path:
    oof_dict[42]  = np.load(seed42_oof_path)
    test_dict[42] = np.load(seed42_test_path)
    print(f"[LOAD] seed42 oof:  {seed42_oof_path}")
    print(f"[LOAD] seed42 test: {seed42_test_path}")
else:
    print("[WARN] seed42 저장 파일을 outputs에서 못찾음. (앙상블은 202/777만으로 계산됨)")

# --- (B) 202,777 학습 ---
for s in RUN_SEEDS:
    print("\n" + "-"*70)
    print(f"Seed {s} run (TE seed={s}, CV seed={s})")
    print("-"*70)

    # seed별 TE 생성 (train_fe/test_fe는 이미 만들어져 있어야 함)
    train_s, test_s = apply_te_for_seed(train_fe, test_fe, seed_for_te=s)

    # inf 방지
    train_s.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_s.replace([np.inf, -np.inf], np.nan, inplace=True)

    res = train_cb_oof(
        train_s, test_s, y,
        feature_cols, cat_cols,
        task_type=TASK_TYPE,
        seed=s,
        cb_params_base=cb_params_base,
        n_folds=N_FOLDS_TRAIN
    )
    results_new.append(res)

    # seed별 저장
    tag_s = f"seed{s}_F{N_FOLDS_TRAIN}_TE{N_FOLDS_TE}_AUC{res['oof_auc']:.6f}"
    oof_path  = os.path.join(OUT_DIR, f"{tag_s}_oof.npy")
    test_path = os.path.join(OUT_DIR, f"{tag_s}_test.npy")
    sub_path  = os.path.join(OUT_DIR, f"{tag_s}_submit.csv")

    np.save(oof_path,  res["oof_pred"])
    np.save(test_path, res["test_pred"])

    sub_s = sub.copy()
    sub_s["probability"] = res["test_pred"]
    sub_s.to_csv(sub_path, index=False)

    print(f"[SAVED seed={s}]")
    print(" -", oof_path)
    print(" -", test_path)
    print(" -", sub_path)

    oof_dict[s]  = res["oof_pred"]
    test_dict[s] = res["test_pred"]

# --- (C) 앙상블 (가능한 seed만 평균) ---
available_seeds = [s for s in ALL_SEEDS_FOR_ENSEMBLE if s in oof_dict]
print("\n[Ensemble] available seeds:", available_seeds)

oof_ens  = np.zeros_like(next(iter(oof_dict.values())))
test_ens = np.zeros_like(next(iter(test_dict.values())))

for s in available_seeds:
    oof_ens  += oof_dict[s]  / len(available_seeds)
    test_ens += test_dict[s] / len(available_seeds)

oof_auc_ens = roc_auc_score(y, oof_ens)
print(f" >>> Ensemble OOF AUC = {oof_auc_ens:.6f}")

# ensemble 저장
tag_ens = f"extra_cb_ens_{oof_auc_ens:.6f}_S{'-'.join(map(str, available_seeds))}_F{N_FOLDS_TRAIN}_TE{N_FOLDS_TE}"

ens_oof_path  = os.path.join(OUT_DIR, f"{tag_ens}_oof.npy")
ens_test_path = os.path.join(OUT_DIR, f"{tag_ens}_test.npy")
ens_sub_path  = os.path.join(OUT_DIR, f"{tag_ens}_submit.csv")
ens_sum_path  = os.path.join(OUT_DIR, f"{tag_ens}_summary.txt")

np.save(ens_oof_path,  oof_ens)
np.save(ens_test_path, test_ens)

sub_ens = sub.copy()
sub_ens["probability"] = test_ens
sub_ens.to_csv(ens_sub_path, index=False)

with open(ens_sum_path, "w", encoding="utf-8") as f:
    f.write("CatBoost Ensemble (partial incremental run)\n")
    f.write("="*70 + "\n")
    f.write(f"DEVICE: {TASK_TYPE}\n")
    f.write(f"SEEDS_USED: {available_seeds}\n")
    f.write(f"N_FOLDS_TRAIN: {N_FOLDS_TRAIN}\n")
    f.write(f"N_FOLDS_TE: {N_FOLDS_TE}\n")
    f.write(f"OOF_AUC_ENSEMBLE: {oof_auc_ens:.6f}\n\n")
    f.write("[cb_params_base]\n")
    f.write(str(cb_params_base) + "\n")

print("\n[ENSEMBLE SAVED]")
print(" -", ens_sub_path)
print(" -", ens_oof_path)
print(" -", ens_test_path)
print(" -", ens_sum_path)


모든 seed 토대로 AUC 가장 높은 조합 찾기

In [ ]:
# ============================================================
# Find Best Seed Ensemble (from saved OOF/Test npy)
# - auto-detect seed*_oof.npy / seed*_test.npy (includes seed42_only*)
# - greedy forward selection to maximize OOF AUC
# - optional: weight tuning on selected seeds (random simplex search)
# - saves best ensemble submit/npy/summary
# ============================================================

import os, re, json, numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

# -------------------------
# Settings
# -------------------------
OUT_DIR = "../outputs"
DATA_DIR = "../data"
TARGET_COL = "임신 성공 여부"   
ID_COL = "ID"

MAX_SEEDS_SELECT = None  # None이면 자동(전부 탐색). 숫자 넣으면 그 개수까지만 greedy로 뽑음
USE_WEIGHT_TUNING = True # False면 단순 평균(1/N)
WEIGHT_TRIALS = 3000     # weight 튜닝 랜덤 탐색 횟수 (빠름)
WEIGHT_SEED = 42         # weight 튜닝 RNG seed

os.makedirs(OUT_DIR, exist_ok=True)

# -------------------------
# Helper: load y, submission
# -------------------------
train_path = os.path.join(DATA_DIR, "train.csv")
sub_path   = os.path.join(DATA_DIR, "sample_submission.csv")

train_df = pd.read_csv(train_path)
y = train_df[TARGET_COL].astype(int).values

sub = pd.read_csv(sub_path)

# -------------------------
# 1) Detect seeds (supports seed42_only_*)
# -------------------------
# examples:
# seed202_F20_TE5_AUC0.740880_oof.npy
# seed42_only_F20_TE5_AUC0.740756_oof.npy
pat = re.compile(r"^seed(\d+)(?:_only)?_F\d+_TE\d+_AUC[\d\.]+_oof\.npy$")

seed_to_paths = {}
for fn in os.listdir(OUT_DIR):
    m = pat.match(fn)
    if not m:
        continue
    seed = int(m.group(1))
    oof_fn = fn
    test_fn = fn.replace("_oof.npy", "_test.npy")
    oof_path = os.path.join(OUT_DIR, oof_fn)
    test_path = os.path.join(OUT_DIR, test_fn)
    if os.path.exists(test_path):
        seed_to_paths[seed] = (oof_path, test_path)

seeds = sorted(seed_to_paths.keys())
print("[FOUND SEEDS]", seeds)

if not seeds:
    raise ValueError("OUT_DIR에서 seed*_oof.npy / seed*_test.npy를 못 찾았어. 경로/파일명 확인!")

# -------------------------
# 2) Load all OOF/Test preds
# -------------------------
oof_dict = {}
test_dict = {}

for s in seeds:
    oof_path, test_path = seed_to_paths[s]
    oof = np.load(oof_path)
    test = np.load(test_path)

    if len(oof) != len(y):
        raise ValueError(f"seed {s} oof 길이({len(oof)}) != y 길이({len(y)})")
    oof_dict[s] = oof.astype(float)
    test_dict[s] = test.astype(float)

# seed별 단일 OOF AUC 출력
single = [(s, roc_auc_score(y, oof_dict[s])) for s in seeds]
single_sorted = sorted(single, key=lambda x: x[1], reverse=True)
print("\n[Single-seed OOF AUC]")
for s, auc in single_sorted:
    print(f" seed={s:<6} auc={auc:.6f}")

# -------------------------
# 3) Greedy Forward Selection (maximize OOF AUC)
# -------------------------
def mean_ensemble_oof(selected):
    arr = np.mean(np.vstack([oof_dict[s] for s in selected]), axis=0)
    return arr

def mean_ensemble_test(selected):
    arr = np.mean(np.vstack([test_dict[s] for s in selected]), axis=0)
    return arr

remaining = set(seeds)
selected = []
best_auc = -1.0
history = []

# 시작 seed: 단일 AUC 1등부터 시작(보통 안정적)
start_seed = single_sorted[0][0]
selected.append(start_seed)
remaining.remove(start_seed)
best_auc = roc_auc_score(y, mean_ensemble_oof(selected))
history.append((selected.copy(), best_auc))
print(f"\n[Greedy] start seed={start_seed}, auc={best_auc:.6f}")

# 몇 개까지 뽑을지
limit = MAX_SEEDS_SELECT if MAX_SEEDS_SELECT is not None else len(seeds)

while len(selected) < limit and len(remaining) > 0:
    cand_best = None
    cand_best_auc = best_auc

    for s in list(remaining):
        cand = selected + [s]
        auc = roc_auc_score(y, mean_ensemble_oof(cand))
        if auc > cand_best_auc + 1e-12:
            cand_best_auc = auc
            cand_best = s

    if cand_best is None:
        print(f"[Greedy] stop: no improvement beyond auc={best_auc:.6f}")
        break

    selected.append(cand_best)
    remaining.remove(cand_best)
    best_auc = cand_best_auc
    history.append((selected.copy(), best_auc))
    print(f"[Greedy] add seed={cand_best} -> auc={best_auc:.6f} | selected={selected}")

best_set = selected
best_oof_mean = mean_ensemble_oof(best_set)
best_test_mean = mean_ensemble_test(best_set)
best_mean_auc = roc_auc_score(y, best_oof_mean)

print("\n[Greedy Result]")
print(" best_set =", best_set)
print(f" best_mean_auc = {best_mean_auc:.6f}")

# -------------------------
# 4) (Optional) Weight tuning on selected seeds
#    Random search on simplex (weights sum to 1, nonnegative)
# -------------------------
def weighted_ensemble(selected, weights, kind="oof"):
    mats = [oof_dict[s] if kind=="oof" else test_dict[s] for s in selected]
    M = np.vstack(mats)   # (k, n)
    w = np.array(weights).reshape(-1, 1)
    return (w * M).sum(axis=0)

best_w = None
best_w_auc = best_mean_auc
best_oof = best_oof_mean
best_test = best_test_mean

if USE_WEIGHT_TUNING and len(best_set) >= 2:
    rng = np.random.default_rng(WEIGHT_SEED)

    # baseline: equal weights
    k = len(best_set)
    w0 = np.ones(k) / k
    oof0 = weighted_ensemble(best_set, w0, "oof")
    auc0 = roc_auc_score(y, oof0)

    print(f"\n[WeightTune] baseline equal weights auc={auc0:.6f}")

    # random simplex weights via Dirichlet
    for t in range(WEIGHT_TRIALS):
        w = rng.dirichlet(alpha=np.ones(k))
        oof_pred = weighted_ensemble(best_set, w, "oof")
        auc = roc_auc_score(y, oof_pred)
        if auc > best_w_auc + 1e-12:
            best_w_auc = auc
            best_w = w
            best_oof = oof_pred
            best_test = weighted_ensemble(best_set, w, "test")

    if best_w is not None:
        print(f"[WeightTune] improved! auc={best_w_auc:.6f}")
        print(" best weights:", {s: float(w) for s, w in zip(best_set, best_w)})
    else:
        print("[WeightTune] no improvement over equal-weight mean.")

# -------------------------
# 5) Save best ensemble artifacts
# -------------------------
used_seeds = best_set
final_auc = best_w_auc if best_w is not None else best_mean_auc
final_oof = best_oof
final_test = best_test
final_weights = (best_w.tolist() if best_w is not None else (np.ones(len(used_seeds))/len(used_seeds)).tolist())

tag = f"BESTENS_S{'-'.join(map(str, used_seeds))}_OOF{final_auc:.6f}"
ens_oof_path  = os.path.join(OUT_DIR, f"{tag}_oof.npy")
ens_test_path = os.path.join(OUT_DIR, f"{tag}_test.npy")
ens_sub_path  = os.path.join(OUT_DIR, f"{tag}_submit.csv")
ens_sum_path  = os.path.join(OUT_DIR, f"{tag}_summary.txt")

np.save(ens_oof_path, final_oof)
np.save(ens_test_path, final_test)

sub_out = sub.copy()
# sample_submission이 probability 컬럼인지 확인해서 맞춰주기
if "probability" in sub_out.columns:
    sub_out["probability"] = final_test
else:
    # 혹시 컬럼명이 다르면 첫 번째 예측 컬럼에 넣음
    pred_col = [c for c in sub_out.columns if c != ID_COL][0]
    sub_out[pred_col] = final_test

sub_out.to_csv(ens_sub_path, index=False)

with open(ens_sum_path, "w", encoding="utf-8") as f:
    f.write("Best Seed Ensemble from saved npy\n")
    f.write("="*70 + "\n")
    f.write(f"OUT_DIR: {OUT_DIR}\n")
    f.write(f"FOUND_SEEDS: {seeds}\n")
    f.write(f"SELECTED_SEEDS: {used_seeds}\n")
    f.write(f"OOF_AUC: {final_auc:.6f}\n")
    f.write(f"WEIGHTED: {best_w is not None}\n")
    f.write(f"WEIGHTS: {final_weights}\n\n")
    f.write("[Greedy History]\n")
    for ss, auc in history:
        f.write(f"{ss} -> {auc:.6f}\n")

print("\n[SAVED BEST ENSEMBLE]")
print(" -", ens_sub_path)
print(" -", ens_oof_path)
print(" -", ens_test_path)
print(" -", ens_sum_path)


BESTENS_S202-1024-4096-42_OOF0.741108_submit.csv 제출